# DPO

## 1.第一问

## 1.1  BT 推导出 Loss function 

首先假设一个先验的 reward model,则 human 对给定 prompt $x$ 及 answer $y$ 的 “喜爱度” (preference) 可由改reward model 计算：

$$
\begin{align}
r^*(y,x)
\end{align}
$$

但我们不知道这个 model 实际长什么样，所以用一些办法来拟合它，如类似于softmax的 BT model ：

$$
\begin{align}
p^*(y_1 \succ y_2 \mid x) = \frac{\exp(r^{*}(x, y_1))}{\exp(r^{*}(x, y_1)) + \exp(r^{*}(x, y_0))}
\end{align}
$$


则根据一个标注好的数据集，我们假设其服从上式 BT model 
$$\mathcal{D}=\{ x^{(i)} , y_w^{(i)}, y_l^{(i)} \}_{i=1}^N$$

则我们可以构造样本概率：

$$
\begin{align}
P(\mathcal{D}) &= \prod_{i=1}^N p^*(y_w^{(i)} \succ y_l^{(i)} \mid x^{(i)}) \\
&= \prod_{i=1}^N \frac{\exp(r^{*}(x^{(i)}, y_w^{(i)}))}{\exp(r^{*}(x^{(i)}, y_w^{(i)})) + \exp(r^{*}(x^{(i)}, y_l^{(i)}))}
\end{align}
$$

进一步， 有

$$
\begin{align}
\ln P(\mathcal{D})
&= \ln \left(\prod_{i=1}^N \frac{\exp(r^{*}(x^{(i)}, y_w^{(i)}))}{\exp(r^{*}(x^{(i)}, y_w^{(i)})) + \exp(r^{*}(x^{(i)}, y_l^{(i)}))} \right)\\
&= \sum_{i=1}^N \ln \left(  \frac{\exp(r^{*}(x^{(i)}, y_w^{(i)}))}{\exp(r^{*}(x^{(i)}, y_w^{(i)})) + \exp(r^{*}(x^{(i)}, y_l^{(i)}))}\right) \\
&= \sum_{i=1}^N \ln \left(  \frac{1}{1 + \exp\left(r^{*}(x^{(i)}, y_l^{(i)}) - r^{*}(x^{(i)}, y_w^{(i)})\right)}\right) \\
\end{align}
$$

设 $\mathcal{R}$ 为所有可能的 $r^*$ 函数集合，则使用MLE， 有

$$
\begin{align}
\max\limits_{r^* \in \mathcal{R}} \ln P(\mathcal{D}) &= \min\limits_{r^* \in \mathcal{R}} \left[ -\ln P(\mathcal{D}) \right] \\
&= \min\limits_{r^* \in \mathcal{R}} - \sum_{i=1}^N \ln \left(  \frac{1}{1 + \exp\left(r^{*}(x^{(i)}, y_l^{(i)}) - r^{*}(x^{(i)}, y_w^{(i)})\right)}\right)
\end{align}
$$


当然，上式不太可能用梯度算出个解析解了，那就把上式改造成一个 Loss function 吧。首先，在理论上表示的话，我们很乐意用一个数学期望来代替那个求和符号，那对于一个特定的 $\hat{r} \in \mathcal{R}$ 我们有:

$$
\begin{align}
\mathcal{L(\hat{r}, \mathcal{D})} = \mathbb{E}\left[- \ln \left(  \frac{1}{1 + \exp\left(\hat{r}(x^{(i)}, y_l^{(i)}) - \hat{r}(x^{(i)}, y_w^{(i)})\right)}\right)\right]
\end{align}
$$


不过在实际里还是多半算个样本平均作为期望的平替就好了

最终，我们就得到一个拟合出来的 reward model $\hat{r}(x,y)$

## 1.2 优化奖励转化为优化概率

如果该题是询问 reward model 是怎么算出 P^* 的话，那么如 BT model 所示其实是用的softmax 函数，很抱歉我有点不太确定题目问的是不是这个意思（）

当然，如果问题询问的是DPO如何使用数学的力量将RLHF中RL一步里使用最大化奖励来求解 $\pi_{\theta}$ 转化为最大化样本似然函数的话，那么请见下面推导：

（请原谅我在此部分偷偷懒，不亲自打latex了，感觉会有点耗时间且收益不大）

首先呢，还是eq3中那个 reward function, 只是我们对它做一点小变形：

![T2_p1](素材\T2_p1.png)

其中最后一个等式怎么得来的，请见我糟糕的手写（想必您已经知道我使用latex的原因了）：

![T2_p2](素材\T2_p2.jpg)

接下来，我们就令分母部分为 $\pi^*$:

![T2_p3](素材\T2_p3.png)

然后神奇的发现这玩意儿确实是个概率，因为它的本质和softmax类似。那么我们就可以把它看作是一个对 $\pi$ 的估计分布喽。接下来进一步对我们的loss funtion化简：

![T2_p4](素材\T2_p4.png)

然后使用一些第二问回答中的内容：KL散度的性质，可以发现当且仅当 $\pi = \pi^*$ 时，上述loss function取得最小值，因此我们就有了一个解析解：

![T2_p5](素材\T2_p5.png)

当然，因为现实情况里可能有很多很多的$y$，导致 Z(x) 的可计算性不高，因此得想办法绕开计算Z(x)，就想了个办法，式子两边同时apply 自然对数，得到

![T2_p6](素材\T2_p6.png)

然后把这个reward model代入到样本的p^* 中，得到

![T2_p7](素材\T2_p7.png)

接着咱就能用MLE的思路来拟合这个 $\pi$ 了

# 2. 第二问

## 2.1 KL散度在 eq3 中的作用

KL散度（反向）在离散和连续情况下的定义式分别如下：

$$
\begin{align}
D_{\text{KL}}(Q \parallel P) = \sum_{i} Q(i) \log \left( \frac{Q(i)}{P(i)} \right)
\end{align}
$$

$$
\begin{align}
D_{\text{KL}}(Q \parallel P) = \int_{-\infty}^{\infty} q(x) \log \left( \frac{q(x)}{p(x)} \right) dx
\end{align}
$$

其中 $P(i),p(x)$ 为概率（或概率密度）实际值， $Q(i), q(i)$ 为估计值

首先来看看上述函数性质（证明从略）：

- $D_{KL} \ge 0$， 当且仅当 p 和 q 几乎处处相等时才有等号成立
- 若要最小化$D_{KL}$，则对于真实分布 $P$ 中概率趋近于0的部分，$Q$ 也得是趋近于或等于0，否则 $\frac{Q}{P} \rightarrow \infty$， 所以最小化反向KL散度得到的估计分布可能会将真实分布中一些稀罕事件的概率拟合为0，然后只在真实分布中概率较大的部分概率非零，简单来说有点像是“抓大放小”

那么我们回到论文里的 eq3：

$$
\begin{align}
\max\limits_{\pi_\theta} \mathbb{E}_{x \sim D, y \sim \pi_{\text{ref}}(y|x)} \left[ \hat{r}(x, y) \right] - \beta D_{\text{KL}} \left[ \pi_{\theta}(y | x) \parallel \pi_{\text{ref}}(y | x) \right]
\end{align}
$$


前半部分 $\mathbb{E}_{x \sim D, y \sim \pi_{\text{ref}}(y|x)}\left[ r_{\phi}(x, y) \right]$ 好理解，就是想把给定prompt $x$时 human对结果 $y$ 的“喜爱度” 的期望最大化，这本来就是我们RLHF的目标。 

有意思的是后半部分 $- \beta D_{\text{KL}} \left[ \pi_{\theta}(y | x) \parallel \pi_{\text{ref}}(y | x) \right]$, 首先， 
$$\min D_{KL} = \max -D_{KL}$$
所以负号没毛病，本质就是想在使喜爱度最大化的同时兼顾$D_{KL}$ 最小化， 关键这个 $D_{\text{KL}} \left[ \pi_{\theta}(y | x) \parallel \pi_{\text{ref}}(y | x) \right]$ 最小化是为了什么。注意到 $\pi_{\text{ref}}(y | x)$ 是没有做 RLHF 前的原始预训练模型对于给定 prompt $x$ 的“结果生成policy”， 而 $\pi_{\theta}(y | x)$ 是我们要找的经调教后的新的“结果生成policy”，那么我们希望这次 RLHF 条件只是在原来预训练模型基础上做一些小微调，并不像翻天覆地的变化，那么这个需求翻译为数学语言就是

**想让  $\pi_{\theta}(y | x)$ 和原来的 $\pi_{\text{ref}}(y | x)$ 这俩分布还是总体上要相似**

那么这就是最小化$D_{\text{KL}} \left[ \pi_{\theta}(y | x) \parallel \pi_{\text{ref}}(y | x) \right]$所能提供的作用了，同时比起正向KL散度的0厌恶特性，反向KL能够将 $\pi_{\text{ref}}(y | x)$ 中一些很小概率的部分拟合为0，算是一种降噪？？，那么现在最大化目标函数就是个多目标优化问题，一面最大化奖励，一面最小化KL散度，根据后者的重要性系数 $\beta$ 在两个目标间找一个平衡。

DPO提出之前更多是用RL对eq3做近似求解。

# 3. 第三问

## 3.1 RL在LLM上的应用

基于本文，RL在LLM的优化上可用于在RLHF中基于估计出的 reward model 构造出最大化目标函数后化为一个reward function，常用的是
$$
\begin{align}
r(x, y) = \hat{r}(x, y) - \beta \left( \log \pi_{\theta}(y \mid x) - \log \pi_{\text{ref}}(y \mid x) \right)
\end{align}
$$
然后用PPO算法来求解得到 $\pi_{\theta}$


## 3.2 RL 是否可以学到Human的偏好reward

根据本文，在RLHF中，RL并非直接根据样本$\mathcal{D}$ 来学习Human的偏好reward，而是实行拿来主义，使用一个根据 最小negative log-likelihood loss 拟合出来的 reward model 来对大模型的$\pi^{SFT}(x)$进行调教

## 3.3 RL是否可以不用预训练就完成对LLM的训练



以我目前浅薄的认知，还没有见过RL取代预训练直接训练LLM的情况，个人理解的原因在于：

1. 预训练是一个构造 $\pi^{SFT}$ 的过程，只有有了 $\pi^{SFT}$, 才有可选的 $y_1, y_2 \sim \pi^{SFT}(y|x)$, 才能进一步用RL去决策选哪一个才能使得现有 reward model 得到的奖励最大化。所以，<span style='color:skyblue'>预训练是从0到1的过程</span>。

2. 而RL是在给出的可选项中做出最大化收益的那个选择，是从1到无穷的过程，但都是基于预训练结果 $\pi^{SFT}$ 去做一些微调。

3. 如果直接让RL来从0到1训练一个LLM，会存在一些问题：首先，从一个随机初始化的参数矩阵（或向量）开始作为第一步，因为其中各个参数 $w_{ij}\in \mathbb{R}$, 所以所有可能的参数组合构成一个无穷大的可行域，RL面对无穷的选择显然很难找到那个能构成有意义人类语言的参数组合，而相比之下，用预训练好的LLM，其基于一个prompt生成的有限个备选 $y$ 就显然给了RL很好的发挥空间，因为RL本质上就是在做利益最大化决策嘛。

# 4. 第四问

![T2_p8](素材\T2_p8.png)

由上表可知：

- DPO 在分布外数据（CNN/DailyMail）上表现优于 PPO，胜率更高。

同时，文中表示，即使没有使用额外的 Reddit TL;DR 未标注数据，DPO 仍然展现出良好的泛化能力。
这说明 DPO 所学到的偏好不仅仅局限于训练数据分布，而是具有一定的迁移能力。